In [1]:
import pandas as pd
import numpy as np
import os

# Configurazione percorsi
BASE_DIR = os.path.dirname(os.getcwd())
DATA_DIR = os.path.join(BASE_DIR, 'dataset')

TRAIN_PATH = os.path.join(DATA_DIR, 'train_step6.csv')
TEST_PATH = os.path.join(DATA_DIR, 'test_step5.csv')
OUTPUT_PATH = os.path.join(DATA_DIR, 'test_step6.csv')

print(f"Reading Train from: {TRAIN_PATH}")
print(f"Reading Test from: {TEST_PATH}")

Reading Train from: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/interim/splits/train_step6.csv
Reading Test from: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/interim/splits/test_step5.csv


In [2]:
def feature_engineering_test(df):
    """
    Applica le stesse trasformazioni del Train al Test set.
    Ricrea le colonne mancanti (aggregazioni, incroci, ratio).
    """
    # Lavoriamo su una copia per non modificare l'originale
    df = df.copy()

    # =================================================================
    # 1. RICREAZIONE GRUPPI DI SINTOMI (Deduotti dalle colonne test)
    # =================================================================

    # Gruppo Respiratorio
    cols_respiratory = [
        'curill_respiratory', 'curill_pneumonia', 'curill_bronchitis',
        'curill_sinus', 'curill_flu', 'history_asthma', 'history_copd'
    ]
    # Filtriamo solo quelle che esistono davvero nel df attuale
    cols_respiratory = [c for c in cols_respiratory if c in df.columns]
    print(f"Respiratory cols found: {len(cols_respiratory)}")

    # Gruppo Cardiaco (Ho messo le storie cliniche cardiache presenti)
    cols_cardiac = [
        'history_cardiac', 'history_heart', 'history_coronary',
        'history_hypertension', 'meds_antihypertensive'
    ]
    cols_cardiac = [c for c in cols_cardiac if c in df.columns]
    print(f"Cardiac cols found: {len(cols_cardiac)}")

    # Gruppo Neurologico
    cols_neuro = [
        'history_neurologic', 'history_stroke', 'history_seizure'
    ]
    cols_neuro = [c for c in cols_neuro if c in df.columns]
    print(f"Neuro cols found: {len(cols_neuro)}")

    # Gruppo Febbre
    cols_fever = ['curill_fever']
    cols_fever = [c for c in cols_fever if c in df.columns]

    # --- CALCOLO I NUMERI (Counts) ---
    df['num_symp_respiratory'] = df[cols_respiratory].sum(axis=1)
    df['num_symp_cardiac']     = df[cols_cardiac].sum(axis=1)
    df['num_symp_neurologic']  = df[cols_neuro].sum(axis=1)
    df['num_symp_fever']       = df[cols_fever].sum(axis=1)

    # --- CALCOLO I FLAG (Booleani: 1 se ha almeno un sintomo, 0 altrimenti) ---
    df['symp_respiratory'] = (df['num_symp_respiratory'] > 0).astype(int)
    df['symp_cardiac']     = (df['num_symp_cardiac'] > 0).astype(int)
    df['symp_neurologic']  = (df['num_symp_neurologic'] > 0).astype(int)
    df['symp_fever']       = (df['num_symp_fever'] > 0).astype(int)

    # --- CALCOLO TOTALE SINTOMI CALCOLATI ---
    # Nota: Usiamo questo come denominatore per i ratio
    df['num_symp_total'] = (
        df['num_symp_respiratory'] +
        df['num_symp_cardiac'] +
        df['num_symp_neurologic'] +
        df['num_symp_fever']
    )

    # =================================================================
    # 2. FEATURE CROSSES (Moltiplicazioni)
    # =================================================================

    # Età x Numero Sintomi (Uso NUMERO_SINTOMI originale se presente, o quello calcolato)
    col_sintomi_base = 'NUMERO_SINTOMI' if 'NUMERO_SINTOMI' in df.columns else 'num_symp_total'

    df['fc_age_x_num_symptoms'] = df['AGE_YRS'] * df[col_sintomi_base]

    # History x Numero Sintomi
    if 'has_history' in df.columns:
        df['fc_history_x_num_symptoms'] = df['has_history'] * df[col_sintomi_base]
    else:
        # Fallback se has_history non c'è (creiamo una somma di history)
        history_cols = [c for c in df.columns if 'history_' in c]
        df['fc_history_x_num_symptoms'] = df[history_cols].max(axis=1) * df[col_sintomi_base]

    # Age x History Cardiac
    # Attenzione: Usiamo 'history_cardiac' se c'è, altrimenti la somma calcolata sopra
    if 'history_cardiac' in df.columns:
        df['fc_age_x_history_cardiac'] = df['AGE_YRS'] * df['history_cardiac']
    else:
        df['fc_age_x_history_cardiac'] = df['AGE_YRS'] * df['symp_cardiac']

    # =================================================================
    # 3. RATIOS (Divisioni)
    # =================================================================

    # Ratio Respiratory
    # Gestiamo la divisione per zero con numpy
    df['ratio_symp_respiratory'] = np.where(
        df['num_symp_total'] > 0,
        df['num_symp_respiratory'] / df['num_symp_total'],
        0
    )

    return df

In [3]:
# Caricamento dataset
train_df = pd.read_csv(TRAIN_PATH)
test_df = pd.read_csv(TEST_PATH)

print(f"Dimensioni Train step 6: {train_df.shape}")
print(f"Dimensioni Test step 5 (originale): {test_df.shape}")

Dimensioni Train step 6: (536370, 79)
Dimensioni Test step 5 (originale): (167616, 66)


In [4]:
# Applicazione Feature Engineering su Test
test_engineered = feature_engineering_test(test_df)

print(f"Dimensioni Test post-processing: {test_engineered.shape}")

Respiratory cols found: 7
Cardiac cols found: 5
Neuro cols found: 3
Dimensioni Test post-processing: (167616, 79)


In [5]:
# Verifica allineamento colonne
train_cols = list(train_df.columns)
test_cols = list(test_engineered.columns)

missing_in_test = [c for c in train_cols if c not in test_cols]
print(f"Colonne mancanti nel Test: {missing_in_test}")

extra_in_test = [c for c in test_cols if c not in train_cols]
print(f"Colonne extra nel Test: {extra_in_test}")

# Riordina colonne Test come in Train
# Se mancano colonne nel test (es. features costanti nel train eliminate?), dovremmo gestirlo.
# Per ora assumiamo che la logica copra tutto.
if not missing_in_test:
    test_final = test_engineered[train_cols]
    print("Test set riordinato e allineato correttamente al Train.")
else:
    print("ATTENZIONE: Impossibile allineare perfettamente. Verifica le colonne mancanti.")
    test_final = test_engineered # Fallback

Colonne mancanti nel Test: []
Colonne extra nel Test: []
Test set riordinato e allineato correttamente al Train.


In [6]:
# Salvataggio
test_final.to_csv(OUTPUT_PATH, index=False)
print(f"Test set salvato in: {OUTPUT_PATH}")

Test set salvato in: /Users/marcodonatiello/PycharmProjects/JupyterProject/data/interim/splits/test_step6.csv
